In [ ]:
import numpy as np
import open3d as o3d
from scipy.interpolate import griddata
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter


# Step 1: Load and Preprocess the Point Cloud

# Load Point Cloud Data from .pcd File
pcd = o3d.io.read_point_cloud(r'')  # Replace with your file path

# Translate the point cloud so that the minimum z-coordinate is at zero
points = np.asarray(pcd.points)
min_z = np.min(points[:, 2])
pcd.translate((0, 0, -min_z))

# print(points)  #sanity checks
# print(points[:, 2])

# Extract x, y, z coordinates
points_clean = np.asarray(pcd.points)
x = points_clean[:, 0]
y = points_clean[:, 1]
z = points_clean[:, 2]

# Visualizing the Point Cloud
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=z,
        colorscale='Viridis',
        opacity=0.8
    )
)])
fig.update_layout(
    title='Cleaned Point Cloud',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)'
    ),
    width=800,
    height=600
)
fig.show()

# Step 2: Create a Regular Grid (DEM)

# Define grid resolution 
grid_resolution = 400  

# Create grid points
xi = np.linspace(x.min(), x.max(), grid_resolution)
yi = np.linspace(y.min(), y.max(), grid_resolution)
xi, yi = np.meshgrid(xi, yi)

# Interpolate z-values onto the grid using linear interpolation
zi = griddata((x, y), z, (xi, yi), method='linear')

# Handle NaN values
zi = np.nan_to_num(zi, nan=0.0)

# Smooth the interpolated grid using a Gaussian filter
sigma = 2  # Adjust sigma to control the amount of smoothing
zi = gaussian_filter(zi, sigma=sigma)

# Visualization of the Smoothed Surface
fig = go.Figure(data=[go.Surface(
    x=xi,
    y=yi,
    z=zi,
    colorscale='Viridis',
    opacity=0.8
)])
fig.update_layout(
    title='Interpolated Surface (DEM)',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)'
    ),
    width=800,
    height=600
)
fig.show()
# Step 3: Visualize Grid Cells as Lines

# Prepare data for grid lines along x-direction
x_lines = []
y_lines = []
z_lines = []

# Lines along x-direction
for i in range(grid_resolution):
    x_lines.extend(xi[i, :])
    y_lines.extend(yi[i, :])
    z_lines.extend(zi[i, :])
    x_lines.append(None)  # Break the line
    y_lines.append(None)
    z_lines.append(None)

# Lines along y-direction
for j in range(grid_resolution):
    x_lines.extend(xi[:, j])
    y_lines.extend(yi[:, j])
    z_lines.extend(zi[:, j])
    x_lines.append(None)  # Break the line
    y_lines.append(None)
    z_lines.append(None)

# Create a 3D line plot of the grid cells
fig2 = go.Figure()

# Add grid lines
fig2.add_trace(go.Scatter3d(
    x=x_lines,
    y=y_lines,
    z=z_lines,
    mode='lines',
    line=dict(color='black', width=1),
    name='Grid Lines'
))

#overlaying the interpolated surface
fig2.add_trace(go.Surface(
    x=xi,
    y=yi,
    z=zi,
    colorscale='Viridis',
    opacity=0.6,
    showscale=False,
    name='Interpolated Surface'
))

fig2.show()

# Step 4: Calculate the Volume Under the Surface

# Calculate the area of each grid cell
dx = (x.max() - x.min()) / (grid_resolution - 1)
dy = (y.max() - y.min()) / (grid_resolution - 1)
cell_area = dx * dy  # Area of each cell in m^2

# Volume = Sum of (cell area * cell height)
total_volume = np.sum(zi * cell_area)  # Volume in cubic meters (m^3)

#Printing outputs
print(f"The total volume under the surface is: {total_volume:.8f} cubic meters")

porosity_vol = total_volume * 0.7 #porosity assumed to be 30% as of now
print(f"The total volume adjusted for porosity: {porosity_vol:.8f} cubic meters")

mass = porosity_vol*7850 #Average density of steel
print(f"The total mass of Balls in the pile: {mass:.4f} KGs")

balls=porosity_vol/0.0010725333 #Volume of a ball in the min at the time
print(f"The total number Balls in the pile: {balls: .4f} Balls")

